In [12]:
%pip install nltk
%pip install underthesea

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Cài đặt thư viện cần thiết
# %pip install nltk
# %pip install pandas

import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Đường dẫn file
file_path = './test_English/dataset_en/Amazon_Product_Review_full_en.csv'
output_path = './test_English/dataset_en/Amazon_Product_Review_full_en_cleaned.csv'

# Đọc dữ liệu, phân tách bằng dấu ;
df = pd.read_csv(file_path, sep=';', dtype=str)

# Loại bỏ các dòng null hoặc chỉ chứa khoảng trắng ở review_body và review_headline
df = df.dropna(subset=['review_body', 'review_headline'])
df = df[~df['review_body'].str.strip().eq('')]
df = df[~df['review_headline'].str.strip().eq('')]
# --- Cân bằng số lượng mẫu giữa hai lớp sentiment ---
if 'sentiment' in df.columns:
    # Đảm bảo sentiment là số
    df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')
    # Loại bỏ các dòng sentiment bị NaN
    df = df.dropna(subset=['sentiment'])
    # Lấy số lượng nhỏ nhất giữa hai lớp
    min_count = df['sentiment'].value_counts().min()
    # Lấy ngẫu nhiên min_count mẫu cho mỗi lớp
    df = df.groupby('sentiment').sample(n=min_count, random_state=42)
    # Shuffle lại dữ liệu
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    print('Số lượng mẫu mỗi lớp sau khi cân bằng:', df['sentiment'].value_counts().to_dict())
# --- Kết thúc cân bằng ---
# Loại bỏ duplicate
df = df.drop_duplicates()
# Loại bỏ null
df = df.dropna()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text_en(text):
    if not isinstance(text, str):
        return ''
    # Loại bỏ thẻ HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Chuyển về chữ thường
    text = text.lower()
    # Loại bỏ ký tự đặc biệt, số, dấu câu
    text = re.sub(r'[^a-z\\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Loại bỏ stopword
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]  # giữ lại từ có độ dài > 1
    # Lemmatize
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return ' '.join(tokens)

# Áp dụng cho review_body và review_headline
df['review_body'] = df['review_body'].astype(str).apply(clean_text_en)
df['review_headline'] = df['review_headline'].astype(str).apply(clean_text_en)

# Loại bỏ các dòng trống sau khi làm sạch
df = df[~df['review_body'].str.strip().eq('')]
df = df[~df['review_headline'].str.strip().eq('')]

# Lưu lại file đã làm sạch
df.to_csv(output_path, sep=';', index=False)
print('Làm sạch hoàn tất. File được lưu tại:', output_path)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Số lượng mẫu mỗi lớp sau khi cân bằng: {0: 5077, 1: 5077}
Làm sạch hoàn tất. File được lưu tại: ./test_English/dataset_en/Amazon_Product_Review_full_en_cleaned.csv
